# Building a Spam Filter with Naive Bayes

In this project, we will build a spam filter with Naive Bayes. We're going to study the practical side of the Naive Bayes algorithm by building a spam filter for SMS messages.

To classify messages as spam or non-spam, the computer would need to follow the below logic:

* Learns how humans classify messages.
* Uses that human knowledge to estimate probabilities for new messages — probabilities for spam and non-spam.
* Classifies a new message based on these probability values — if the probability for spam is greater, then it classifies the message as spam. Otherwise, it classifies it as non-spam (if the two probability values are equal, then we may need a human to classify the message).

So our first task is to "teach" the computer how to classify messages. To do that, we'll use the multinomial Naive Bayes algorithm along with a dataset of 5,572 SMS messages that are already classified by humans.

The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the The [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). You can also download the dataset directly [from this link](https://dq-content.s3.amazonaws.com/433/SMSSpamCollection). The data collection process is described in more details on [this page](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition), where you can also find some of the authors' papers.

Let's start by reading in the dataset. You'll be able to find the solutions to this project at this link or by clicking the key icon at the top right of the interface.

**Disclaimer:** This project is part of series of guided projects on dataquest. All of the code and logic belongs to me with [dataquest](dataquest.io) providing guidance on building the spam filter. The project is built as a result of going through the coded classes on the platform. Formulas and content is taken from the guided project.

## Data Description

Let's read in the dataset and try to get acquainted with it.

In [1]:
# load necessary libraries
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split

In [2]:
# read in the dataset with no header
data = pd.read_csv("SMSSpamCollection", sep='\t', header=None, names=['Label', 'SMS'])

In [3]:
# check few rows 
data.head(2)

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...


In [4]:
# columns and rows in the dataset
print("Dataset contains {} rows and {} columns".format(*data.shape))

Dataset contains 5572 rows and 2 columns


In [5]:
data.Label.value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

Our dataset is somewhat inbalanced since we have 86,5% of ham and only 13,4% of spam messages. Nevertheless, let's move to preparing our dataset.

## Preparing Data

Let's start by splitting our data in training and test set. We will first randomize the entire dataset to ensure that ham and spam messages are spread properly throughout the dataset.

In [6]:
# randomize the entire dataset
data = data.sample(frac=1, random_state=1)

# split the data to train and test
training_data, testing_data = train_test_split(data)

In [7]:
# check percentage of ham and spam in training set
training_data.Label.value_counts(normalize=True)

ham     0.865997
spam    0.134003
Name: Label, dtype: float64

As we can see, the proprtion of ham and spam in the training set is similar to the original dataset.

### Reminder on Naive Bayes

Let's recall from our previous classes and missions that when a new message comes in, our Naive Bayes algorithm will make the classification based on the results it gets to these two equations (note that we replaced "Spam$^C$" with "Ham" inside the second equation below):

\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam)
\end{equation}

\begin{equation}
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}

Also, to calculate P(wi|Spam) and P(wi|Ham) inside the formulas above, recall that we need to use these equations:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
\end{equation}

\begin{equation}
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

Let's also summarize what the terms in the equations above mean:

\begin{aligned}
&N_{w_i|Spam} = \text{the number of times the word } w_i \text{ occurs in spam messages} \\
&N_{w_i|Spam^C} = \text{the number of times the word } w_i \text{ occurs in non-spam messages} \\
\\
&N_{Spam} = \text{total number of words in spam messages} \\
&N_{Spam^C} = \text{total number of words in non-spam messages} \\
\\
&N_{Vocabulary} = \text{total number of words in the vocabulary} \\
&\alpha = 1 \ \ \ \ (\alpha \text{ is a smoothing parameter})
\end{aligned}

To calculate probabilities we will need to transform our dataset to a format where can calculate them. Basically each word in the messages will become a feature with each row representing the counts of occurence of that word in the specific row/message. We will transform the SMS column to vector of words.

Let's transform the SMS column to vector of words and relevant counts.

In [8]:
# let's make a copy of our training data to avoid SettingWithCopyWarning
training_set = training_data.copy(deep=True)

# lower string and remove punctuation and numbers from SMS column
training_set['SMS'] = training_data['SMS'].str.replace('[^a-zA-Z]', ' ').str.lower()

# check results
training_set.head()

,Label,SMS
2739,ham,i sent you the prices and do you mean the lt...
4437,ham,house maid is the murderer coz the man was mu...
2259,ham,sad story of a man last week was my b day m...
3482,ham,wherre s my boytoy
4067,ham,fyi i m gonna call you sporadically starting a...


In [9]:
# create a vocabulary for the messages in the training set

# get the list of all words
all_words = [word for lst in training_set['SMS'].str.split().values for word in lst]

# get the vocabulary
vocabulary = list(set(all_words))

In [10]:
# set up a dictionary with words as keys and counts as columns
word_counts_per_sms = {unique_word: [0] * len(training_set['SMS']) for unique_word in vocabulary}

# loop through and populate the dictionary with counts
for index, sms in enumerate(training_set['SMS'].str.split()):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [11]:
# convert dictionary to dataframe
sms_word_vectors = pd.DataFrame(word_counts_per_sms)

# reset index for training set
training_set.reset_index(inplace=True, drop=True)

# concatenate the dataframe with traininng data
training_data = pd.concat([sms_word_vectors, training_set], axis=1)

In [12]:
# check our final training data
training_data.head(3)

,a,aa,aah,aaniye,aathi,abbey,abel,aberdeen,abi,ability,...,zf,zhong,zindgi,zoe,zogtorius,zoom,zs,zyada,Label,SMS
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,ham,i sent you the prices and do you mean the lt...
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,ham,house maid is the murderer coz the man was mu...
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,ham,sad story of a man last week was my b day m...


## Calcualting Constants First

Now that we have the word vector representation of each of our messages. Let's recall that the Naive Bayes algorithm will need to know the probability values of the two equations below to be able to classify new messages:

\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam) \\
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}

\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam) \\
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}

Also, to calculate P(wi|Spam) and P(wi|Ham) inside the formulas above, recall that we need to use these equations:
\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}} \\
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}} \\
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

We will also need to introduce a parameter $\alpha$ which essentially prevents division by 0 where probabilities are 0, so called Laplace Smoothing.

Let's first calculate:
* P(Spam) and P(Ham)
* $N_{spam}$, $N_{ham}$, $N_{vocabulary}$

In [13]:
# initialize an alpha paramter
alpha = 1

# calculate overall probabilities
p_spam = training_data['Label'].value_counts(normalize=True)[1]
p_ham = training_data['Label'].value_counts(normalize=True)[0]

# calculate counts of words in spam, ham and vocab
n_spam = training_data.loc[training_data.Label == 'spam', training_data.columns.difference(['SMS', 'Label'])].sum().sum()
n_ham = training_data.loc[training_data.Label == 'ham', training_data.columns.difference(['SMS', 'Label'])].sum().sum()
n_vocab = len(vocabulary)

## Calculating Parameters

Now, we will use our transformed training data to calculate the probabilities for each word ocurring given the label of the message. For instance, let's say we receive two new messages:
* "secret code"
* "secret party 2night"

We'll need to calculate P("secret"|Spam) for both these messages, and we can use the training set to get the values we need to find a result for the equation below:

\begin{equation}
P(\text{"secret"}|Spam) = \frac{N_{"secret"|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
\end{equation}

We will perform the calculation for both types of messages, spam and ham. A generalized formula for our case would be like the following:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}} \\
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

In [14]:
# initialize for each type of message a dictionary(vocabulary) with key being words from vocabulary, and values set to 0
spam_words = {key: 0 for key in vocabulary}
ham_words = {key: 0 for key in vocabulary}

# isolate spam and ham messages from our training data
spam_training_set = training_data[training_data['Label'] == 'spam']
ham_training_set = training_data[training_data['Label'] == 'ham']

# iterate over vocabularies of spam and ham words and calculate probabilities using formulas above
for word in spam_words:
    word_count = spam_training_set[str(word)].sum()
    word_prob = (word_count + alpha) / (n_spam + alpha * n_vocab)
    spam_words[word] = word_prob

for word in ham_words:
    word_count = ham_training_set[word].sum()
    word_prob = (word_count + alpha) / (n_ham + alpha * n_vocab)
    ham_words[word] = word_prob

## Classifying a new message

Now that we've calculated all the constants and parameters we need, we can start creating the spam filter. The filter can be understood as 

* a function that takes as an input list of words
* calculates the probability of spam or ham given the list of words
* compares the values of those probabilities
* returns the label for which the probability is higher
    * in case of equality, then human intervention would be needed to classify the message
    
To write the function we need to use these 2 equations:

\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam)
\end{equation}

In [15]:
# function to classify the message as spam or ham
def classify(message: str):

    # perform clean up of input message and split to words (tokenization)
    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()
    
    # calculate probability of spam/ham given message
    p_spam_given_message = p_spam * np.prod([spam_words[w] for w in message if w in spam_words])
    p_ham_given_message = p_ham * np.prod([ham_words[w] for w in message if w in ham_words])

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

Let's test our classifier with dummy data.

In [16]:
test_message1 = 'WINNER!! This is the secret code to unlock the money: C3421.'
test_message2 = "Sounds good, Tom, then see u there"

print("Classifying message 1")
classify(test_message1)
print("\n")
print("Classifying message 2")
classify(test_message2)

Classifying message 1
P(Spam|message): 2.9659964898116258e-25
P(Ham|message): 3.0485737687547568e-27
Label: Spam


Classifying message 2
P(Spam|message): 8.288309333372596e-25
P(Ham|message): 4.969856371695636e-21
Label: Ham


It seems that our classifier was able to correctly classify these two test messages.

## Measuing the Spam Filter's Accuracy

We will now use our test set, prepared earlier to test the accuracy of our spam filter. For that we need to slightly modify our initial classify function to output spam and ham instead of printing results.

In [17]:
# function to classify the message as spam or ham
def classify_test_set(message: str):

    # perform clean up of input message and split to words (tokenization)
    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()
    
    # calculate probability of spam/ham given message
    p_spam_given_message = p_spam * np.prod([spam_words[w] for w in message if w in spam_words])
    p_ham_given_message = p_ham * np.prod([ham_words[w] for w in message if w in ham_words])

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'needs human classification'

Now that we have a function that returns labels instead of printing them, we can use it to create a new column in our test set.

In [18]:
testing_data['predicted'] = testing_data['SMS'].apply(classify_test_set)
testing_data.head()

/dataquest/system/env/python3/lib/python3.4/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Label,SMS,predicted
2605,ham,You call times job today ok umma and ask them ...,ham
5114,ham,Argh why the fuck is nobody in town ;_;,ham
3243,ham,Good Morning my Dear........... Have a great &...,ham
3641,ham,He's really into skateboarding now despite the...,ham
2799,ham,Purity of friendship between two is not about ...,ham


Now we can compare the predicted values with the actual values to measure how good our spam filter is with classifying new messages. To make the measurement, we'll use accuracy as a metric:

\begin{equation}
\text{Accuracy} = \frac{\text{number of correctly classified messages}}{\text{total number of classified messages}}
\end{equation}

In [24]:
# count the number of correctly classified vs total classified
correctly_classified = testing_data[testing_data.Label == testing_data.predicted].shape[0]
total_classifier = testing_data.shape[0]

# calculate accuracy
accuracy = correctly_classified / total_classifier
print("Accuracy of our classifier is {:.2%}".format(accuracy))

Accuracy of our classifier is 98.35%


## Conclusion & Further Ideas

In this project, we managed to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. The filter had an accuracy of 98.35% on the test set, which is an excellent result.

We can keep working on the classifier to improve it. Here are some ideas that we could take and try to improve the classifier.

* Isolate the 14 messages that were classified incorrectly and try to figure out why the algorithm reached the wrong conclusions.
* Make the filtering process more complex by making the algorithm sensitive to letter case.